In [51]:
import pandas as pd
from sklearn.model_selection import cross_val_score, RepeatedKFold, cross_val_predict
import numpy as np

from xgboost import XGBRegressor

In [84]:
train_x = pd.read_csv("../results/train_x.csv")
train_y = pd.read_csv("../results/train_y.csv")
imdb_info = pd.read_csv("../results/imdb_results.csv")

In [85]:
all_info = train_x.merge(imdb_info, left_on="player_1_name", right_on="Movie", how = "left")

In [86]:
all_info = all_info.merge(imdb_info, left_on="player_2_name", right_on="Movie", suffixes = ("_player1", "_player2"), how="left")

In [87]:
all_info = all_info.drop(columns=["player_1_name", "player_2_name", "Movie_player1", "Movie_player2", "id_player1", "id_player2"])

In [88]:
xgb_model = XGBRegressor(n_estimators=500)

results = cross_val_score(xgb_model, 
                 X = all_info,
                 y = train_y["plus_minus"],
                 scoring="neg_mean_squared_error",
                 cv=RepeatedKFold()
               )
np.sqrt(abs(results)).mean()

19.55040441678845

In [89]:
results = cross_val_predict(xgb_model, 
                 X = all_info,
                 y = train_y["plus_minus"],
                 cv=5
               )

In [90]:
train_y["predicted"] = results

In [91]:
train_y["correct"] = np.sign(train_y["plus_minus"]) * np.sign(train_y["predicted"])

In [92]:
train_y.loc[train_y["correct"] == -1, :].to_csv("../results/mistakes1.2.csv", index = False)

In [93]:
train_y.to_csv("../results/gen_1.2_model.csv", index = False)

In [94]:
from eli5 import show_weights

In [99]:
xgb_model.fit(all_info, train_y["plus_minus"])

show_weights(xgb_model, top=(5,5))

TypeError: '>=' not supported between instances of 'tuple' and 'int'

In [97]:
?show_weights

Signature: show_weights(estimator, **kwargs)
Docstring:
Return an explanation of estimator parameters (weights)
as an IPython.display.HTML object. Use this function
to show classifier weights in IPython.

:func:`show_weights` accepts all
:func:`eli5.explain_weights` arguments and all
:func:`eli5.formatters.html.format_as_html`
keyword arguments, so it is possible to get explanation and
customize formatting in a single call.

Parameters
----------
estimator : object
    Estimator instance. This argument must be positional.

top : int or (int, int) tuple, optional
    Number of features to show. When ``top`` is int, ``top`` features with
    a highest absolute values are shown. When it is (pos, neg) tuple,
    no more than ``pos`` positive features and no more than ``neg``
    negative features is shown. ``None`` value means no limit.

    This argument may be supported or not, depending on estimator type.

target_names : list[str] or {'old_name': 'new_name'} dict, optional
    Names of 